In [2]:
from cmj.loa import models
from cmj.utils import Manutencao, run_sql
from django.conf import settings
from django.db.models import Q, F
from django.db.models.aggregates import Sum
from decimal import Decimal
import fitz
import re
m = Manutencao()
m.desativa_auto_now()
m.desativa_signals()

In [3]:
loa = models.Loa.objects.get(pk=2021)

In [7]:
(models.Despesa.objects.filter(loa=loa).delete(), 
models.Orgao.objects.filter(loa=loa).delete(),
models.UnidadeOrcamentaria.objects.filter(loa=loa).delete(),
models.Funcao.objects.filter(loa=loa).delete(),
models.SubFuncao.objects.filter(loa=loa).delete(),
models.Programa.objects.filter(loa=loa).delete(),
models.Acao.objects.filter(loa=loa).delete(),
models.Natureza.objects.filter(loa=loa).delete(),
models.EmendaLoaRegistroContabil.objects.filter(emendaloa__loa=loa).delete())

((0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {'loa.EmendaLoaRegistroContabil': 0}))

In [13]:
autografo = NormaJuridica.objects.get(pk=8841)
autografo.texto_integral.url

'/api/norma/normajuridica/8841/texto_integral/'

In [7]:
dfile = fitz.open(autografo.texto_integral.path)
#dfile = fitz.open('/home/leandro/Downloads/anexo_materia.pdf')

In [8]:
def load_pages_with(page_mask=''):
    rtext = ''
    for np in range(len(dfile)):
        page = dfile.load_page(np)
        text = page.get_text("text")

        if page_mask not in text:
            continue

        rtext += text
    while '  ' in rtext:
        rtext = rtext.replace('  ', ' ')
    while ' \n' in rtext:
        rtext = rtext.replace(' \n', '\n')
    while '\n\n' in rtext:
        rtext = rtext.replace('\n\n', '\n')
    return rtext

In [9]:
text_global = load_pages_with(page_mask='Quadro de Detalhamento da Despesa\nEXERCÍCIO DE 2021\n')

In [14]:
len(text_global), text_global[0:1000]

(48398,
 'ESTADO DE GOIÁS\nMUNICÍPIO DE JATAI\nQuadro de Detalhamento da Despesa\nEXERCÍCIO DE 2021\nRUA ITARUMA, 355 - S. SANTA MARIA\n1\nPÁGINA\nUNIDADE ORÇAMENTÁRIA\nÓRGÃO\nCÓDIGO\nCÓDIGO\nAPLICAÇÃO PROGRAMADA\nCÓDIGOS\nDETALHADA\nAPLICADO\nNATUREZA DA\nCLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\nCAMARA MUNICIPAL DE JATAI\nCAMARA MUNICIPAL\n01\n01\nOBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL\n01.031.0139.1.001\n4.4.90.51.00\n218.196,31\n4.4.90.52.00\n453.360,24\n671.556,55\nMANUT.ATIV.DA CAMARA MUNICIPAL\n01.031.0139.2.001\n3.1.90.04.00\n2.000,00\n3.1.90.11.00\n11.635.818,60\n3.1.90.16.00\n2.000,00\n3.1.90.92.00\n100,00\n3.3.90.14.00\n85.000,00\n3.3.90.30.00\n513.000,00\n3.3.90.33.00\n1.000,00\n3.3.90.34.00\n4.000,00\n3.3.90.35.00\n1.000,00\n3.3.90.36.00\n160.000,00\n3.3.90.37.00\n1.000,00\n3.3.90.39.00\n800.000,00\n3.3.90.40.00\n150.000,00\n3.3.90.92.00\n100,00\n13.355.018,60\nENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA\n01.271.2839.9.061\n3.1.90.13.00\n1.365.000,00\n3.1.90.54.00\n10

In [15]:
aplicacoes = []
for m in re.finditer(r'(.+)\n(\d{2}\.\d{3}\.\d{4}\.\d\.\d{3})', text_global):
    aplicacoes.append([m.start(), m.end(), list(m.groups())])
aplicacoes.append([len(text_global),len(text_global),None])
for i in range(len(aplicacoes)-1):
    aplic = aplicacoes[i]
    x1 = aplicacoes[i][1]
    x2 = aplicacoes[i+1][0]

    natdespesas = []

    for n in re.finditer(r'(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(\d{0,3}(?:\.\d{3})*,\d{2})', text_global[x1:x2]):
        natdespesas.append(list(n.groups()))

    aplic.append(natdespesas)
    
aplicacoes.pop()
aplicacoes = list(map(lambda x: (x[2][1], x[2][0], x[0], x[1], tuple(x[3])), aplicacoes))

In [16]:
aplicacoes

[('01.031.0139.1.001',
  'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
  329,
  382,
  (['4.4.90.51.00', '218.196,31'], ['4.4.90.52.00', '453.360,24'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  442,
  490,
  (['3.1.90.04.00', '2.000,00'],
   ['3.1.90.11.00', '11.635.818,60'],
   ['3.1.90.16.00', '2.000,00'],
   ['3.1.90.92.00', '100,00'],
   ['3.3.90.14.00', '85.000,00'],
   ['3.3.90.30.00', '513.000,00'],
   ['3.3.90.33.00', '1.000,00'],
   ['3.3.90.34.00', '4.000,00'],
   ['3.3.90.35.00', '1.000,00'],
   ['3.3.90.36.00', '160.000,00'],
   ['3.3.90.37.00', '1.000,00'],
   ['3.3.90.39.00', '800.000,00'],
   ['3.3.90.40.00', '150.000,00'],
   ['3.3.90.92.00', '100,00'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  823,
  881,
  (['3.1.90.13.00', '1.365.000,00'],
   ['3.1.90.54.00', '100,00'],
   ['3.1.90.56.00', '100,00'],
   ['3.1.90.58.00', '100,00'],
   ['3.1.90.92.00', '100,00'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PR

In [17]:
unidades_temp = []
for n in re.finditer(r'(?:CLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\n)(.+)\n(.+)\n(.+)\n(.+)\n', text_global):
    unidades_temp.append([n.start(), n.end(), list(n.groups())])
    
unidades_temp.append([len(text_global),len(text_global),None])

In [18]:
unidades_temp[0]

[231, 329, ['CAMARA MUNICIPAL DE JATAI', 'CAMARA MUNICIPAL', '01', '01']]

In [19]:
unidades = []
for i in range(len(unidades_temp)-1):
    unidade = unidades_temp[i]
    x1 = unidades_temp[i][1]
    x2 = unidades_temp[i+1][0]
    
    unidade = [(unidade[2][2], unidade[2][0]), (unidade[2][3], unidade[2][1]), []]
    for aplic in aplicacoes:
        if aplic[2] >= x1 and aplic[3] <= x2:
            unidade[-1].append(aplic)    
    unidades.append(unidade)    

In [20]:
unidades_temp = unidades
unidades = []

orgao = ''
unidade_orc = ''
for u in unidades_temp:
    if u[0] != orgao or u[1] != unidade_orc:
        orgao = u[0]
        unidade_orc = u[1]
        unidades.append([orgao, unidade_orc, []])
    unidades[-1][-1].extend(u[2])

In [21]:
unidades

[[('01', 'CAMARA MUNICIPAL DE JATAI'),
  ('01', 'CAMARA MUNICIPAL'),
  [('01.031.0139.1.001',
    'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
    329,
    382,
    (['4.4.90.51.00', '218.196,31'], ['4.4.90.52.00', '453.360,24'])),
   ('01.031.0139.2.001',
    'MANUT.ATIV.DA CAMARA MUNICIPAL',
    442,
    490,
    (['3.1.90.04.00', '2.000,00'],
     ['3.1.90.11.00', '11.635.818,60'],
     ['3.1.90.16.00', '2.000,00'],
     ['3.1.90.92.00', '100,00'],
     ['3.3.90.14.00', '85.000,00'],
     ['3.3.90.30.00', '513.000,00'],
     ['3.3.90.33.00', '1.000,00'],
     ['3.3.90.34.00', '4.000,00'],
     ['3.3.90.35.00', '1.000,00'],
     ['3.3.90.36.00', '160.000,00'],
     ['3.3.90.37.00', '1.000,00'],
     ['3.3.90.39.00', '800.000,00'],
     ['3.3.90.40.00', '150.000,00'],
     ['3.3.90.92.00', '100,00'])),
   ('01.271.2839.9.061',
    'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
    823,
    881,
    (['3.1.90.13.00', '1.365.000,00'],
     ['3.1.90.54.00', '100,00'],
     ['3.1.90.56.00', '100,0

# Funções e Subfunções

In [22]:
with open(f'funcao.txt', 'r') as fnd:
    funcoes = fnd.read()
    funcoes = re.split(r'\||\n', funcoes)
    funcoes = [(funcoes[i], funcoes[i+1]) for i in range(0, len(funcoes)-1, 2)]
    for c, e in funcoes:
        created, f = models.Funcao.objects.get_or_create(loa=loa, codigo=c, especificacao=e)
funcoes = sorted(funcoes, key=lambda x: (x[0], x[1]))
models.SubFuncao.objects.filter(loa=loa).count()

0

In [23]:
with open(f'subfuncao.txt', 'r') as fnd:
    subfs = fnd.read()
    subfs = re.split(r'\t|\n', subfs)
    subfs = [(subfs[i], subfs[i+1], subfs[i+2]) for i in range(0, len(subfs)-2, 3)]
    for fc, c, e in subfs:
        f = models.Funcao.objects.filter(loa=loa, codigo=fc).first()
        created, sf = models.SubFuncao.objects.get_or_create(loa=loa, funcao=f, codigo=c, especificacao=e)
subfs = sorted(subfs, key=lambda x: (x[0], x[1]))
models.SubFuncao.objects.filter(loa=loa).count()

114

In [24]:
count = [0,0,0]
for orgao, unidade, despesas in unidades:
    count[0] += 1
    for desp, espec, x1, x2, naturezas in despesas:
        count[1] += 1
        for nat in naturezas:
            count[2] += 1
print(count)

[28, 161, 742]


In [25]:
count = 0
for orgao, unidade, despesas in unidades:
    o_obj, created = models.Orgao.objects.get_or_create(
        loa=loa, codigo=orgao[0], especificacao=orgao[1]
    )

    uo_obj, created = models.UnidadeOrcamentaria.objects.get_or_create(
        loa=loa, orgao=o_obj, codigo=unidade[0], especificacao=unidade[1]
    )
    
    for desp, espec, x1, x2, naturezas in despesas:
        desp_re = re.match(r'(?P<funcao>\d{2})\.(?P<subfuncao>\d{3})'
                           r'\.(?P<programa>\d{4})\.(?P<acao>\d\.\d{3})', desp)

        funcao_obj = models.Funcao.objects.filter(loa=loa, codigo=desp_re['funcao']).first()
        subfunc_obj = models.SubFuncao.objects.filter(loa=loa, codigo=desp_re['subfuncao']).first()
        
        programa_obj, created = models.Programa.objects.get_or_create(loa=loa, codigo=desp_re['programa'])
        
        acao_obj, created = models.Acao.objects.get_or_create(loa=loa, codigo=desp_re['acao'], especificacao=espec)
        
        for nat in naturezas:
            count += 1
            natureza, created = models.Natureza.objects.get_or_create(loa=loa, codigo=nat[0])
            
            desp_obj, created = models.Despesa.objects.get_or_create(
                loa = loa,
                orgao = o_obj,
                unidade = uo_obj,
                funcao = funcao_obj,
                subfuncao = subfunc_obj,
                programa = programa_obj,
                acao = acao_obj,
                natureza = natureza)
            
            desp_obj.valor_materia = Decimal(nat[1].replace('.', '').replace(',', '.'))
            
            try: 
                desp_obj.save()
            except Exception as e:
                print(e)
                break
        if not desp_obj.pk:
            break
print(count, models.Despesa.objects.filter(loa=loa).count())

742 742


In [27]:
a = """text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.15 DE 20/06/78')
naturezas = []
for m in re.finditer(r'\n?(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n([^0-9]*)?\n?', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])
for esp, cod in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()
    """

# Natureza da Despesa

In [28]:
text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.08 DE 04/02/85')
naturezas = []
for m in re.finditer(r'\n(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(.*)\n', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])

In [29]:
for cod, esp in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()

In [30]:
with open(f'2024_natureza_despesa.txt', 'r') as fnd:
    lines = fnd.read()
    lines = re.split(r'\t|\n', lines)
    naturezas = [(lines[i], lines[i+1]) for i in range(0, len(lines)-1, 2)] 
    for cod, esp in naturezas:
        nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
        if not nat.especificacao:
            nat.especificacao=esp
            nat.save() 

# Programas

In [54]:
text_global = load_pages_with(page_mask='Adendo VI a Portaria SOF N.08 de 04/02/85')

In [55]:
programas = []
for m in re.finditer(r'(.+)\n(.+)\n(\d{2}\.\d{3}\.(\d{4}))\n', text_global):
    programas.append(m.groups())
programas

[('PROCESSO LEGISLATIVO', '14.026.575,15', '01.031.0139', '0139'),
 ('ENCARGOS ESPECIAIS', '1.365.400,00', '01.271.2839', '2839'),
 ('ENCARGOS ESPECIAIS', '600.500,00', '01.272.2839', '2839'),
 ('ENCARGOS ESPECIAIS', '1.000,00', '01.845.2839', '2839'),
 ('ENCARGOS ESPECIAIS', '6.524,85', '01.846.2839', '2839'),
 ('APOIO AO PODER JUDICIÁRIO', '7.000,00', '02.061.0239', '0239'),
 ('ADMINISTRACÃO CONECTADA COM O FUTURO',
  '17.156.146,46',
  '04.122.0439',
  '0439'),
 ('ADMINISTRACÃO CONECTADA COM O FUTURO',
  '5.100.900,00',
  '04.123.0439',
  '0439'),
 ('ADMINISTRACÃO CONECTADA COM O FUTURO', '316.100,00', '04.124.0439', '0439'),
 ('ADMINISTRACÃO CONECTADA COM O FUTURO', '75.100,00', '04.422.0439', '0439'),
 ('ENCARGOS ESPECIAIS', '2.000,00', '04.422.2839', '2839'),
 ('ENCARGOS ESPECIAIS', '67.433,98', '04.845.2839', '2839'),
 ('ENCARGOS ESPECIAIS', '110.000,00', '04.846.2839', '2839'),
 ('PROGRAMA DE DEFESA SOCIAL E SEGURANCA PUBLICA',
  '6.768.999,31',
  '06.181.0639',
  '0639'),
 ('A

In [52]:
len(programas), models.Programa.objects.filter(loa=loa).values_list('codigo', flat=True).count()

(76, 22)

In [53]:
for p in list(programas):
    prog, created = models.Programa.objects.get_or_create(loa=loa, codigo=p[-1])
    prog.especificacao=p[0]
    prog.save()